# 1. Preliminaries

In [30]:
##!pip3 install qutip
import numpy as np
import qutip as qp
import scipy as si
import pandas as pd
from random import random, choice, randint

# 2. Gell_Mann matrix generator function

this function will generate the gellmann matrix of dimension d.

the function "gellmann(j,k,d)" will generate the jth_kth gellmann matrix of dimension d. so for every d dimension we have d^2 gellmann matrix. one of them is I, so the number of gellmann matrix that is not trivial is (d^2 - 1).

In [31]:
"""Generate generalized Gell-Mann matrices.
  .. module:: gellmann.py
     :synopsis: Generate generalized Gell-Mann matrices
  .. moduleauthor:: Jonathan Gross <jarthurgross@gmail.com>
"""
from itertools import product

def gellmann(j, k, d):
    r"""Returns a generalized Gell-Mann matrix of dimension d.
    
    According to the convention in *Bloch Vectors for Qubits* by Bertlmann and
    Krammer (2008), returns :math:`\Lambda^j` for :math:`1\leq j=k\leq d-1`,
    :math:`\Lambda^{kj}_s` for :math:`1\leq k<j\leq d`, :math:`\Lambda^{jk}_a`
    for :math:`1\leq j<k\leq d`, and :math:`I` for :math:`j=k=d`.
    Parameters
    ----------
    j : positive integer
        Index for generalized Gell-Mann matrix
    k : positive integer
        Index for generalized Gell-Mann matrix
    d : positive integer
        Dimension of the generalized Gell-Mann matrix
    Returns
    -------
    numpy.array
        A genereralized Gell-Mann matrix.
    """

    if j > k:
        gjkd = np.zeros((d, d), dtype=np.complex128)
        gjkd[j - 1][k - 1] = 1
        gjkd[k - 1][j - 1] = 1
    elif k > j:
        gjkd = np.zeros((d, d), dtype=np.complex128)
        gjkd[j - 1][k - 1] = -1.j
        gjkd[k - 1][j - 1] = 1.j
    elif j == k and j < d:
        gjkd = np.sqrt(2/(j*(j + 1)))*np.diag([1 + 0.j if n <= j
                                               else (-j + 0.j if n == (j + 1)
                                                     else 0 + 0.j)
                                               for n in range(1, d + 1)])
    else:
        gjkd = np.diag([1 + 0.j for n in range(1, d + 1)])

    return gjkd

def get_basis(d):
    r"""Return a basis of operators.
    
    The basis is made up of orthogonal Hermitian operators on a Hilbert space
    of dimension d, with the identity element in the last place.
    Parameters
    ----------
    d : int
        The dimension of the Hilbert space.
    Returns
    -------
    list of numpy.array
        The basis of operators.
    """
    return [gellmann(j, k, d) for j, k in product(range(1, d + 1), repeat=2)]

in this cell the gellmann matrix of 3 dimension will generate and they will save in a dictionary and labeled them by numbers 0,1,...,8 .

In [32]:
def gellmann3():
    gellmann3_basis = {}

    k=0
    for i in range(1,4):
        for j in range(1,4):
            gellmann3_basis.update({"G{}".format(k):gellmann(i,j,3)})
            k+=1
        
    return gellmann3_basis

# 3. Coefficients of gellmann's expansion

at this cell a random density matrix will generate by qutip library.

for calculating the coefficients of gellmann expansion we do this iteration:
1. calculate the tensor product of gellmann matrices for every coefficient
2. calculate the dot product of "ro" and the "gellmann's tensor product"
3. calculate the trace of this matrix and divide it into 4
4. append it to our index list

In [33]:
def coef_of_gellmann(ro):

    coef = {}  ## the coefficient's dictionary (suitable format for saving as a data)
    gellmann3_basis = gellmann3() ## the gellmann matrix for a 3 dimension system
    
    for i in range(9):
        for j in range(9):
            tensorP = np.kron(gellmann3_basis["G{}".format(i)],gellmann3_basis["G{}".format(j)]) ## Tensor product of gelllmann matrices
            c = np.trace(np.dot(ro,tensorP))/4   ## dot product and trace
            c = c.real       ## all of gellmann coefs is a real
            c = round(c,8)  
            coef.update({"a{}{}".format(i,j):c})  ## update the coef dict by a[i][j] and its gellmann coef for g[i][j]

    index = np.array(coef)  ## change list to np.array() object
    return coef

this cell is just a test of this algotithm in 3 dimension bipatite systems.

In [34]:
N = 3

ro = qp.rand_dm(N*N,0.5,dims=[[N,N], [N,N]])
coef_of_gellmann(ro)

{'a00': 0.02889158,
 'a01': -0.00716668,
 'a02': 0.0118049,
 'a03': 0.02391696,
 'a04': -0.03866722,
 'a05': -0.00724331,
 'a06': -0.01932456,
 'a07': -0.00152142,
 'a08': -9.07e-05,
 'a10': 0.00427787,
 'a11': -0.03307099,
 'a12': 0.01424805,
 'a13': -0.00896902,
 'a14': -0.02523914,
 'a15': -0.03098832,
 'a16': 0.01261644,
 'a17': 0.00702582,
 'a18': 0.00712854,
 'a20': -0.0219865,
 'a21': 0.00051913,
 'a22': -0.00145502,
 'a23': 0.0122872,
 'a24': -0.04343986,
 'a25': 0.00281084,
 'a26': 0.00250141,
 'a27': 1.904e-05,
 'a28': 0.00464027,
 'a30': -0.00757164,
 'a31': -0.00896902,
 'a32': -0.0081772,
 'a33': 0.03307099,
 'a34': -0.0037573,
 'a35': 0.02559514,
 'a36': 0.00780561,
 'a37': -0.04225208,
 'a38': -0.02921555,
 'a40': -0.01491319,
 'a41': 0.0187873,
 'a42': 0.00204367,
 'a43': -0.00191495,
 'a44': -0.025742,
 'a45': -0.00339612,
 'a46': -0.00292747,
 'a47': 0.03585477,
 'a48': 0.01494592,
 'a50': -0.02072359,
 'a51': 0.02469402,
 'a52': -0.01917037,
 'a53': -0.04735614,
 'a5

# 4. Generate entangled states (PPT)

In this cell we define a PPT funcion that check positive partial transpose.

In [40]:
def PPT(ro):
    """ positive partial transpose 
    input : density matrix
    out put:list of  coefficient of gellmann matrices and detect entangeled states
    """

    ro_pt = qp.partial_transpose(ro,[0,1])  ## partial transpose in subsystem 2
    eig = ro_pt.eigenstates()   ## calculate the eigenvalues and eigenstates
    eigv = [round(i,8) for i in eig[0]] 
    
    result = coef_of_gellmann(ro)
    
    if (eigv[0]<0 ):
            result.update({"label":1})  ## the entangled states that labeled by 1
            return result
    else :
            result.update({"label":2})  ## the unknown states that labeled by 2
            return result


# 5. Generate separable states (Convex combination)

in this cell we define two function that:
1. Convex_Combination: this function is give a matrix list and return a convex combination of this matrices
2. generate_ran_den_list: this function is generate a list of random density matrices

In [41]:
def Convex_Combination(matrix_list, index=10):
    
    coef = np.random.dirichlet(np.ones(index),size=1)   ## generating list of numbers that is positive and its sum is 1 with length equal to 'index'
    
    seprable_state = qp.Qobj()    ## build an empty Qobj for cycle
    for i in range(index):
        a = choice(matrix_list)   ## choice a density matrix randomly
        b = choice(matrix_list)   ## choice a density matrix randomly
        
        tensorP = np.kron(a,b) ## Tensor product of density matrices
        tensorP *= round(coef[0][i],9)  ## product of tensorP and a coef that come from the coef list
        
        seprable_state += tensorP  ## at the last its a convex combination of tensor product of density matrices
    
    return seprable_state


def generate_ran_den_list(dim, index= 100):
    
    result = []
    for i in range(index):
        density = round(random(),4)     ## for build a random number between 0 and 1

        rand = qp.rand_dm(dim, density)   ## generate a random density matrix with dimension N
        result.append(rand)        ## appending this random density matrix to ran_den_mat list    
    
    return result

# 6. Generate data

in this cell we build 100 000 random states and labeled that. after that we build a data frame for saving and using in future.

In [47]:
%%time

states={} #states
dim = 3   ## dimension of a system

##generate the entangled states
for i in range (1,1000+1):
    density = round(random(),4)     ## for build a random number between 0 and 1
    
    ro = qp.rand_dm(N*N, density, dims=[[N,N], [N,N]])     #generating random density matrix with a random density
    states.update({"{}".format(i):PPT(ro)})     # update the states by its number and label and gellmann coefs


##generate the separable states
ran_den_mat = generate_ran_den_list(dim, 1000) ## a list of random density matrix for a system

for i in range(1000+1,2000+1):
    index = randint(10,100)   ## build a number between 10 and 100 for using in function
    sep = Convex_Combination(ran_den_mat, index)  ## generate a random seprable density matrix
    
    sep_coef = coef_of_gellmann(sep)   ## calculate the gellmann coefs
    sep_coef.update({"label":0})   ## labeled seprable states by 0
    
    states.update({"{}".format(i):sep_coef})  ## update sep_states
    
    
df = pd.DataFrame(data=states).T  # build a data frame by pandas
df

Wall time: 42.9 s


,a00,a01,a02,a03,a04,a05,a06,a07,a08,a10,...,a80,a81,a82,a83,a84,a85,a86,a87,a88,label
1,0.013419,0.031459,0.009614,-0.015459,0.026811,0.019905,0.001276,-0.009955,-0.001210,-0.019392,...,0.053116,0.011226,-0.000368,0.004184,-0.017016,-0.003722,0.028079,-0.010359,0.25,1.0
2,-0.125000,0.000000,0.000000,0.000000,0.072169,0.000000,0.000000,0.000000,0.125000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.144338,0.000000,0.000000,0.000000,0.25,2.0
3,-0.017045,-0.022506,-0.032291,-0.024906,0.011330,0.021429,-0.015495,-0.006474,-0.032585,-0.010296,...,-0.039361,-0.005226,0.053323,0.011726,-0.076253,-0.018187,0.030085,0.000830,0.25,1.0
4,-0.019961,-0.024134,0.006409,-0.005290,-0.000742,0.002531,-0.003593,0.026779,0.032151,0.008560,...,0.000835,-0.064884,-0.009654,0.034657,0.007180,0.021349,-0.026810,0.012322,0.25,1.0
5,0.011517,-0.000197,-0.005013,-0.006538,0.010825,-0.020601,0.014316,-0.005538,-0.009638,0.031151,...,0.016850,-0.028703,0.002436,0.026272,0.032378,0.015507,0.016949,0.011087,0.25,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,0.000346,-0.001516,-0.000920,0.006295,-0.008890,0.003442,-0.003756,0.002432,0.007462,-0.001769,...,0.002109,0.014943,0.013736,-0.006280,-0.023719,0.005030,0.003185,-0.004390,0.25,0.0
1997,0.021592,0.002996,-0.001581,0.000447,0.013401,-0.002816,0.001886,0.001250,0.015490,0.004321,...,0.040320,-0.002378,0.002996,0.013165,0.013777,0.006891,0.017774,-0.009189,0.25,0.0
1998,-0.002240,0.003666,0.002393,-0.003546,0.003419,0.000892,-0.000754,0.001926,-0.015121,0.004109,...,0.018036,-0.006511,-0.015521,0.004273,0.001520,-0.000074,0.008944,-0.009088,0.25,0.0
1999,-0.017121,-0.006763,-0.001015,-0.002520,0.003164,0.001423,0.000692,0.007914,0.037314,-0.002468,...,0.000892,-0.006532,0.001034,-0.004821,-0.028609,0.001075,0.005753,0.005892,0.25,0.0


in this cell we save our data frame as a zip file

In [19]:
df.to_csv('data_of_states.csv.zip')
##df.to_csv('data_of_states.csv')